In [95]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import random

In [107]:
from tensorflow.keras.layers import Dense, Activation, Conv2D,Conv1D, Flatten,MaxPooling2D,BatchNormalization,Lambda, AveragePooling2D, MaxPooling1D
import tensorflow.keras.backend as K
from tensorflow.keras.layers import LSTM,Input,Bidirectional,Dense, Conv1D,MaxPooling1D,Flatten,Dropout,Input
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow import keras
import tensorflow as tf

In [97]:
df_X = pd.read_csv('dataset/train_features.csv')
df_y = pd.read_csv('dataset/train_target.csv')

In [98]:
def data_processing(data_frame,feature_list,conc_list=list()):
    for feature in feature_list:
        conc_list.append(np.asarray(data_frame[feature]).reshape(-1,375,1))
    conc_array = np.concatenate(conc_list,axis=2)
    array = conc_array.transpose(0,1,2)
    return array

In [99]:
data = data_processing(df_X,['S1','S2','S3','S4'])

In [100]:
y = df_y.drop(['id','X','Y','V'],axis=1)
y_numpy = y.to_numpy()
y_list = y_numpy.tolist()

In [101]:
x_array = data
y_array = y_list
train_test_index = int(len(x_array)*0.8)
zip_list = list(zip(x_array,y_array))
random.shuffle(zip_list)
x_array,y_array = zip(*zip_list)
x_train,y_train = np.array(x_array[0:train_test_index]),np.array(y_array[0:train_test_index])
x_test,y_test = np.array(x_array[train_test_index:]),np.array(y_array[train_test_index:])

In [125]:
from sklearn.preprocessing import MinMaxScaler
Scaler = MinMaxScaler(feature_range=(0, 1))
y_train_scaled = Scaler.fit_transform(y_train)
y_test_scaled = Scaler.fit_transform(y_test)
y_train_scaled

array([[0.83333333],
       [0.5       ],
       [1.        ],
       ...,
       [1.        ],
       [0.5       ],
       [0.16666667]])

In [103]:
x_train.shape


(2240, 375, 4)

In [114]:
model= Sequential(
    [
        Input(shape=(x_train.shape[1],x_train.shape[2])),
        # Use a Rescaling layer to make sure input values are in the [0, 1] range.
        # The original images have shape (28, 28), so we reshape them to (28, 28, 1)
        # Follow-up with a classic small convnet
        Conv1D(32, 2, activation="relu"),
        BatchNormalization(),
        MaxPooling1D(3),
        Conv1D(32,2, activation="relu"),
        MaxPooling1D(2),
        Conv1D(32,2, activation="relu"),
        Flatten(),
        Dense(64, activation="relu"),
        Dense(16, activation="relu"),
        Dense(1)
    ]
)

In [115]:
def my_loss(y_true, y_pred):
    return K.mean(K.square(y_true-y_pred))/2e+04

In [116]:
model.compile(loss=my_loss,optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              metrics=['mean_absolute_error'])

In [117]:
from keras import callbacks
early_stopping = callbacks.EarlyStopping(monitor ="val_mean_absolute_error", 
                                        mode ="min", patience = 50, 
                                        restore_best_weights = True)

In [118]:
model.fit(x_train,y_train,validation_split=0.2,epochs=600,callbacks=[early_stopping])

Epoch 1/600
56/56 [==============================] - 2s 17ms/step - loss: 0.6031 - mean_absolute_error: 97.9109 - val_loss: 0.4116 - val_mean_absolute_error: 77.4121
Epoch 2/600
56/56 [==============================] - 1s 13ms/step - loss: 0.3759 - mean_absolute_error: 72.4049 - val_loss: 0.1671 - val_mean_absolute_error: 45.1627
Epoch 3/600
56/56 [==============================] - 1s 13ms/step - loss: 0.1502 - mean_absolute_error: 43.4128 - val_loss: 0.1294 - val_mean_absolute_error: 42.8070
Epoch 4/600
56/56 [==============================] - 1s 13ms/step - loss: 0.1165 - mean_absolute_error: 40.2459 - val_loss: 0.1143 - val_mean_absolute_error: 40.4844
Epoch 5/600
56/56 [==============================] - 1s 14ms/step - loss: 0.1089 - mean_absolute_error: 39.2324 - val_loss: 0.1108 - val_mean_absolute_error: 39.8595
Epoch 6/600
56/56 [==============================] - 1s 15ms/step - loss: 0.1075 - mean_absolute_error: 39.5504 - val_loss: 0.1082 - val_mean_absolute_error: 39.4270
Epoc

Epoch 50/600
56/56 [==============================] - 1s 26ms/step - loss: 0.0100 - mean_absolute_error: 10.1074 - val_loss: 0.0100 - val_mean_absolute_error: 9.6273
Epoch 51/600
56/56 [==============================] - 2s 27ms/step - loss: 0.0087 - mean_absolute_error: 9.3990 - val_loss: 0.0113 - val_mean_absolute_error: 10.8629
Epoch 52/600
56/56 [==============================] - 1s 27ms/step - loss: 0.0096 - mean_absolute_error: 9.9542 - val_loss: 0.0095 - val_mean_absolute_error: 9.5678
Epoch 53/600
56/56 [==============================] - 1s 26ms/step - loss: 0.0086 - mean_absolute_error: 9.4211 - val_loss: 0.0091 - val_mean_absolute_error: 9.3049
Epoch 54/600
56/56 [==============================] - 2s 28ms/step - loss: 0.0095 - mean_absolute_error: 9.9988 - val_loss: 0.0089 - val_mean_absolute_error: 9.1390
Epoch 55/600
56/56 [==============================] - 1s 26ms/step - loss: 0.0077 - mean_absolute_error: 8.8490 - val_loss: 0.0093 - val_mean_absolute_error: 9.8158
Epoch 56

56/56 [==============================] - 1s 16ms/step - loss: 0.0030 - mean_absolute_error: 5.8440 - val_loss: 0.0026 - val_mean_absolute_error: 4.9028
Epoch 149/600
56/56 [==============================] - 1s 16ms/step - loss: 0.0028 - mean_absolute_error: 5.6545 - val_loss: 0.0050 - val_mean_absolute_error: 7.8149
Epoch 150/600
56/56 [==============================] - 1s 16ms/step - loss: 0.0035 - mean_absolute_error: 6.3596 - val_loss: 0.0023 - val_mean_absolute_error: 4.7403
Epoch 151/600
56/56 [==============================] - 1s 17ms/step - loss: 0.0026 - mean_absolute_error: 5.5005 - val_loss: 0.0024 - val_mean_absolute_error: 4.6700
Epoch 152/600
56/56 [==============================] - 1s 16ms/step - loss: 0.0043 - mean_absolute_error: 6.5711 - val_loss: 0.0024 - val_mean_absolute_error: 4.8825
Epoch 153/600
56/56 [==============================] - 1s 16ms/step - loss: 0.0034 - mean_absolute_error: 6.0841 - val_loss: 0.0025 - val_mean_absolute_error: 4.8787
Epoch 154/600
56/5

56/56 [==============================] - 2s 34ms/step - loss: 0.0029 - mean_absolute_error: 5.7806 - val_loss: 0.0019 - val_mean_absolute_error: 4.5933- loss: 0.0029 - mean_absolute_error: 5.7
Epoch 246/600
56/56 [==============================] - 1s 24ms/step - loss: 0.0019 - mean_absolute_error: 4.7997 - val_loss: 0.0015 - val_mean_absolute_error: 4.2364
Epoch 247/600
56/56 [==============================] - 1s 25ms/step - loss: 0.0021 - mean_absolute_error: 4.8807 - val_loss: 0.0025 - val_mean_absolute_error: 5.1516
Epoch 248/600
56/56 [==============================] - 1s 20ms/step - loss: 0.0022 - mean_absolute_error: 5.0788 - val_loss: 0.0015 - val_mean_absolute_error: 4.1102
Epoch 249/600
56/56 [==============================] - 1s 20ms/step - loss: 0.0028 - mean_absolute_error: 5.5460 - val_loss: 0.0013 - val_mean_absolute_error: 3.5144: 0s - loss: 0.0036 - mean_absolute_err
Epoch 250/600
56/56 [==============================] - 1s 19ms/step - loss: 0.0022 - mean_absolute_error

Epoch 294/600
56/56 [==============================] - 1s 17ms/step - loss: 0.0022 - mean_absolute_error: 5.0179 - val_loss: 0.0011 - val_mean_absolute_error: 3.4324
Epoch 295/600
56/56 [==============================] - 1s 16ms/step - loss: 0.0022 - mean_absolute_error: 4.9948 - val_loss: 0.0021 - val_mean_absolute_error: 4.8731
Epoch 296/600
56/56 [==============================] - 1s 16ms/step - loss: 0.0023 - mean_absolute_error: 5.1397 - val_loss: 0.0013 - val_mean_absolute_error: 3.9281
Epoch 297/600
56/56 [==============================] - 1s 16ms/step - loss: 0.0023 - mean_absolute_error: 5.1607 - val_loss: 0.0010 - val_mean_absolute_error: 3.3026
Epoch 298/600
56/56 [==============================] - 1s 16ms/step - loss: 0.0016 - mean_absolute_error: 4.2152 - val_loss: 0.0011 - val_mean_absolute_error: 3.3548
Epoch 299/600
56/56 [==============================] - 1s 16ms/step - loss: 0.0018 - mean_absolute_error: 4.3718 - val_loss: 0.0012 - val_mean_absolute_error: 3.2600
Epoc

56/56 [==============================] - 1s 23ms/step - loss: 0.0013 - mean_absolute_error: 3.7278 - val_loss: 0.0011 - val_mean_absolute_error: 3.2508
Epoch 393/600
56/56 [==============================] - 1s 22ms/step - loss: 0.0015 - mean_absolute_error: 4.0738 - val_loss: 0.0011 - val_mean_absolute_error: 3.1990
Epoch 394/600
56/56 [==============================] - 1s 20ms/step - loss: 0.0016 - mean_absolute_error: 4.1063 - val_loss: 0.0018 - val_mean_absolute_error: 4.5046
Epoch 395/600
56/56 [==============================] - 1s 19ms/step - loss: 0.0012 - mean_absolute_error: 3.7155 - val_loss: 0.0011 - val_mean_absolute_error: 3.2752
Epoch 396/600
56/56 [==============================] - 1s 18ms/step - loss: 0.0012 - mean_absolute_error: 3.6577 - val_loss: 0.0015 - val_mean_absolute_error: 4.0657
Epoch 397/600
56/56 [==============================] - 1s 17ms/step - loss: 0.0014 - mean_absolute_error: 3.9658 - val_loss: 0.0011 - val_mean_absolute_error: 3.5708
Epoch 398/600
56/5

56/56 [==============================] - 1s 15ms/step - loss: 0.0014 - mean_absolute_error: 4.0851 - val_loss: 8.9693e-04 - val_mean_absolute_error: 2.9452
Epoch 489/600
56/56 [==============================] - 1s 18ms/step - loss: 0.0016 - mean_absolute_error: 4.1941 - val_loss: 8.5294e-04 - val_mean_absolute_error: 2.7777
Epoch 490/600
56/56 [==============================] - 1s 17ms/step - loss: 0.0014 - mean_absolute_error: 4.0030 - val_loss: 0.0021 - val_mean_absolute_error: 5.1989
Epoch 491/600
56/56 [==============================] - 1s 16ms/step - loss: 0.0015 - mean_absolute_error: 4.2744 - val_loss: 9.0030e-04 - val_mean_absolute_error: 2.8515
Epoch 492/600
56/56 [==============================] - 1s 16ms/step - loss: 0.0016 - mean_absolute_error: 4.3900 - val_loss: 8.7848e-04 - val_mean_absolute_error: 2.7783
Epoch 493/600
56/56 [==============================] - 1s 16ms/step - loss: 0.0014 - mean_absolute_error: 3.8283 - val_loss: 8.8572e-04 - val_mean_absolute_error: 2.818

Epoch 537/600
56/56 [==============================] - 1s 17ms/step - loss: 0.0017 - mean_absolute_error: 4.6056 - val_loss: 8.9922e-04 - val_mean_absolute_error: 3.0614
Epoch 538/600
56/56 [==============================] - 1s 15ms/step - loss: 0.0012 - mean_absolute_error: 3.6459 - val_loss: 0.0011 - val_mean_absolute_error: 3.3194
Epoch 539/600
56/56 [==============================] - 1s 17ms/step - loss: 0.0012 - mean_absolute_error: 3.7512 - val_loss: 0.0012 - val_mean_absolute_error: 3.4527


# 25 to 175

In [140]:
model.evaluate(x_test,y_test)

18/18 [==============================] - 0s 4ms/step - loss: 0.0010 - mean_absolute_error: 2.9951


[0.0010429472895339131, 2.995065450668335]

In [65]:
ans = model.predict(np.expand_dims(x_test[0],axis=0))